Model

In [305]:
# Import libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

Settings

In [306]:
catalog_lunar_dir = "./data/lunar/data/training/catalogs/apollo12_catalog_GradeA_final.csv"
data_dir = "./data/lunar/data/training/data/Processed"
chunk_size = 20000

Enter Raw Test File Inof:


In [307]:
raw_data_dir = "./data/lunar/data/training/data/Processed"
file_name = "xa.s12.00.mhz.1974-04-08HR00_evid00141.csv"

Functions

In [308]:
#Process single lunar data

def process_lunar_data(catalog_path, data_dir, row_num, chunk_size):
    # Read the catalog file
    catalog_lunar = pd.read_csv(catalog_path)

    # Extract row and relevant data
    row = catalog_lunar.iloc[row_num]
    arrival_time = datetime.strptime(row["time_abs(%Y-%m-%dT%H:%M:%S.%f)"], '%Y-%m-%dT%H:%M:%S.%f')
    arrival_time_relative = row["time_rel(sec)"]
    test_filename = row.filename

    # Read the associated CSV file containing raw data
    csv_file = f'{data_dir}/{test_filename}.csv'
    raw_data = pd.read_csv(csv_file)

    # Initialize the results dataframe
    total_rows = raw_data.shape[0]
    results_df = pd.DataFrame(columns=["chunk", "label"])

    # Iterate over chunks of data
    start = 0
    while start < total_rows:
        end = min(start + chunk_size, total_rows)  # Handle case where we don't have a full chunk at the end
        chunk = raw_data.iloc[start:end]
        data_df = pd.DataFrame(chunk["time_rel(sec)"])

        # Check if arrival time is within the current chunk
        if arrival_time_relative >= data_df["time_rel(sec)"].values.min() and arrival_time_relative <= data_df["time_rel(sec)"].values.max():
            # Split chunk at the arrival_time_relative
            before_arrival = chunk[data_df["time_rel(sec)"] < arrival_time_relative]
            after_arrival = chunk[data_df["time_rel(sec)"] >= arrival_time_relative]

            # Add the part before arrival_time_relative with label 0
            if not before_arrival.empty:
                results_df = pd.concat([results_df, pd.DataFrame({"chunk": [before_arrival], "label": [0]})], ignore_index=True)

            # Add a chunk starting from the arrival_time_relative, ensure it has chunk_size rows
            after_start = after_arrival.index[0]  # Start from the first row after the arrival time
            after_end = min(after_start + chunk_size, total_rows)  # Ensure the chunk has exactly chunk_size rows
            after_chunk = raw_data.iloc[after_start:after_end]

            results_df = pd.concat([results_df, pd.DataFrame({"chunk": [after_chunk], "label": [1]})], ignore_index=True)

            # Move the start index beyond this chunk (chunk_size after the arrival time)
            start = after_end
        else:
            # If no arrival time in this chunk, label the entire chunk as 0
            results_df = pd.concat([results_df, pd.DataFrame({"chunk": [chunk], "label": [0]})], ignore_index=True)
            start += chunk_size  # Move to the next chunk

    # If there are remaining rows less than the chunk size, add them with the appropriate label
    if start < total_rows:
        remaining_chunk = raw_data.iloc[start:total_rows]
        remaining_label = 1 if arrival_time_relative >= remaining_chunk["time_rel(sec)"].values.min() else 0
        results_df = pd.concat([results_df, pd.DataFrame({"chunk": [remaining_chunk], "label": [remaining_label]})], ignore_index=True)

    return results_df







# Process entire catalog

def process_entire_catalog(catalog_dir, data_dir, chunk_size):
    try:
        catalog_lunar = pd.read_csv(catalog_dir)
        testing_df = pd.DataFrame(columns=["chunk", "label"])  # Initialize an empty dataframe to store results
        
        # Iterate over every row in the catalog
        for row_num in range(len(catalog_lunar)):
            # Process each row using process_lunar_data and append the result to testing_df
            results_df = process_lunar_data(catalog_dir, data_dir, row_num, chunk_size)
            testing_df = pd.concat([testing_df, results_df], ignore_index=True)
    
    except FileNotFoundError:
        print("File not found")
    
    return testing_df






# extracting the features

def extract_features_from_chunk(chunk):
    features = {}
    
    
    
    # Velocity features
    # features['mean_velocity'] = chunk["velocity(m/s)"].mean()
    
    # features['max_velocity'] = chunk["velocity(m/s)"].max()
    # features['min_velocity'] = chunk["velocity(m/s)"].min()
    # features['std_velocity'] = chunk["velocity(m/s)"].std()
    # features['range_velocity'] = features['max_velocity'] - features['min_velocity']
    
    # # Energy features
    # features['total_energy'] = (chunk["velocity(m/s)"] ** 2).sum()
    # features['rms_velocity'] = np.sqrt((chunk["velocity(m/s)"] ** 2).mean())
    
    # # Frequency/Oscillation features (zero crossings)
    zero_crossings = np.where(np.diff(np.sign(chunk["velocity(m/s)"])))[0]
    features['zero_crossing_rate'] = len(zero_crossings)
    
    
    
    return features





# converting to suitable

def convert_to_polished_df(testing_df):
    polished_data = []

    for idx, row in testing_df.iterrows():
        chunk = row["chunk"]
        label = row["label"]
        
        # Extract statistical features from chunk
        features = extract_features_from_chunk(chunk)
        features['label'] = label  # Add the label for quake/no-quake
        
        # Append to polished data
        polished_data.append(features)

    # Convert to a new DataFrame
    polished_df = pd.DataFrame(polished_data)
    return polished_df





# Process raw input into ready for model to read
def process_raw_file_for_model(raw_data_dir, file_name, chunk_size):
    # Path to the CSV file
    csv_file = f"{raw_data_dir}/{file_name}"
    
    # Read the raw data from the CSV file
    raw_data = pd.read_csv(csv_file)
    total_rows = raw_data.shape[0]

    # Lists to store statistics for each chunk and the chunks themselves
    stats_list = []
    chunks_list = []  # List to store the chunks

    # Iterate over chunks of data in the file
    start = 0
    while start < total_rows:
        end = min(start + chunk_size, total_rows)
        chunk = raw_data.iloc[start:end]

        # Store the current chunk in the chunks list
        chunks_list.append(chunk)

        # Calculate statistical values from the chunk
        # mean_velocity = chunk['velocity(m/s)'].mean()
        
        # max_velocity = chunk['velocity(m/s)'].max()
        # min_velocity = chunk['velocity(m/s)'].min()
        # std_velocity = chunk['velocity(m/s)'].std()
        # range_velocity = max_velocity - min_velocity
        # total_energy = (chunk['velocity(m/s)'] ** 2).sum()
        # rms_velocity = (chunk['velocity(m/s)'] ** 2).mean() ** 0.5

        # Calculate zero crossing rate
        zero_crossings = np.where(np.diff(np.sign(chunk["velocity(m/s)"])))[0]
        zero_crossing_rate = len(zero_crossings)  # Count of zero crossings


        # Add the calculated statistics to the stats list
        stats_list.append({
            # 'mean_velocity': mean_velocity,
            
            # 'max_velocity': max_velocity,
            # 'min_velocity': min_velocity,
            # 'std_velocity': std_velocity,
            # 'range_velocity': range_velocity,
            # 'total_energy': total_energy,
            # 'rms_velocity': rms_velocity,
            'zero_crossing_rate': zero_crossing_rate
        })

        # Move to the next chunk
        start += chunk_size

    # Create DataFrames from the lists
    stats_df = pd.DataFrame(stats_list)  # DataFrame for statistics
    chunks_df = pd.DataFrame({'chunks': chunks_list})  # DataFrame for chunks

    return stats_df, chunks_df


# Plot Input Data

def plot_data(data_dir, file_name):
    
    test_filename = file_name
    csv_file = f'{data_dir}/{test_filename}'
    data = pd.read_csv(csv_file)

    csv_times = np.array(data["time_rel(sec)"].tolist())
    csv_velocity = np.array(data["velocity(m/s)"].tolist())

    # Plot the trace!
    fig,ax = plt.subplots(1,1,figsize=(10,3))
    ax.plot(csv_times,csv_velocity)
    # Make the plot pretty
    ax.set_xlim([min(csv_times),max(csv_times)])
    ax.set_ylabel('Velocity (m/s)')
    ax.set_xlabel('Time (s)')
    ax.set_title(f'{test_filename}', fontweight='bold')
    

Training Data

In [309]:
testing_chunks = process_entire_catalog(catalog_lunar_dir, data_dir, chunk_size)
polished_df = convert_to_polished_df(testing_chunks)

label_counts = polished_df["label"].value_counts()
print(label_counts)

count_label_1 = label_counts.get(1, 0)

count_label_1

File not found
label
0    571
1     20
Name: count, dtype: int64


20

In [310]:
polished_df

zero_crossing_rate  label
0                     0      0
1                     0      0
2                    12      0
3                     0      0
4                     0      0
..                  ...    ...
586                   0      0
587                   0      0
588                  22      0
589                   0      0
590                   0      0

[591 rows x 2 columns]

Model Implementation

In [311]:
#Option 1 - Using LR

from sklearn.linear_model import LogisticRegression

# Split the dataset into features (X) and target labels (y)
X = polished_df.drop(columns=['label'])  # Features
y = polished_df['label']  # Target labels (1 for quake, 0 for non-quake)


# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize the Random Forest Classifier
clf = LogisticRegression( max_iter=1000, random_state=42)

# Train the model
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))





Accuracy: 0.9898648648648649
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99       286
           1       0.77      1.00      0.87        10

    accuracy                           0.99       296
   macro avg       0.88      0.99      0.93       296
weighted avg       0.99      0.99      0.99       296



In [312]:
# Option 2 - Balancing the data out more

df = polished_df

# Split the DataFrame into two separate DataFrames for each class
df_majority = df[df['label'] == 0]
df_minority = df[df['label'] == 1]

# Specify the number of majority class instances you want to keep
# Here we keep the same number as minority class instances
n_minority = len(df_minority)
df_majority_undersampled = df_majority.sample(n=n_minority, random_state=42)  # Randomly sample from majority class

# Combine the undersampled majority class with the minority class
df_balanced = pd.concat([df_majority_undersampled, df_minority])

# Shuffle the DataFrame to mix class labels
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Now `df_balanced` is your new training dataset with a balanced class distribution

# Split the dataset into features (X) and target labels (y)
X = df_balanced.drop(columns=['label'])  # Features
y = df_balanced['label']  # Target labels (1 for quake, 0 for non-quake)


# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
clf2 = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
clf2.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf2.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))




Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         3

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



In [313]:
# Option 3 - USED

# Split the dataset into features (X) and target labels (y)
X = polished_df.drop(columns=['label'])  # Features
y = polished_df['label']  # Target labels (1 for quake, 0 for non-quake)

# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize the Random Forest Classifier
clf3 = RandomForestClassifier(class_weight="balanced", random_state=42)

# Train the model
clf3.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf3.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9898648648648649
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99       286
           1       0.82      0.90      0.86        10

    accuracy                           0.99       296
   macro avg       0.91      0.95      0.93       296
weighted avg       0.99      0.99      0.99       296



In [314]:
f"{raw_data_dir}/{file_name}"

'./data/lunar/data/training/data/Processed/xa.s12.00.mhz.1974-04-08HR00_evid00141.csv'

In [315]:
ready_testing_df, chuncks_testing_df = process_raw_file_for_model(raw_data_dir, file_name, chunk_size)

In [316]:
chuncks_testing_df

chunks
0          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_re...
1          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_re...
2          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_re...
3          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_re...
4          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_re...
5           time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
6           time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
7           time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
8           time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
9           time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
10          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
11          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
12          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
13          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
14          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
15          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
16          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
17          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
18          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
19          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
20          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
21          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
22          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
23          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
24          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
25          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
26          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
27          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...
28          time_abs(%Y-%m-%dT%H:%M:%S.%f)  time_r...

In [317]:
ready_testing_df

zero_crossing_rate
0                    0
1                    0
2                    0
3                    0
4                    0
5                    0
6                    0
7                    0
8                    0
9                    0
10                1566
11                   6
12                  10
13                  12
14                  24
15                  14
16                  56
17                   2
18                   0
19                   0
20                   0
21                   0
22                  12
23                   0
24                   0
25                   0
26                   0
27                  14
28                  18

In [318]:
predicted_labels = clf3.predict(ready_testing_df)
predicted_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [319]:
indices_of_ones = np.where(predicted_labels == 1)[0]
indices_of_ones

array([10], dtype=int64)

Important Chunks

In [320]:
def plot_chunk(chunk):
    csv_times = np.array(chunk["time_rel(sec)"].tolist())
    csv_velocities = np.array(chunk["velocity(m/s)"].tolist())

    # Plot the trace!
    fig,ax = plt.subplots(1,1,figsize=(10,3))
    ax.plot(csv_times,csv_velocities)
    # Make the plot pretty
    ax.set_xlim([min(csv_times),max(csv_times)])
    ax.set_ylabel('Velocity (m/s)')
    ax.set_xlabel('Time (s)')

for index in indices_of_ones:
    chunk_plot = chuncks_testing_df["chunks"].values[index]
    plot_chunk(chunk_plot)

Whole Data Set

In [321]:
plot_data(raw_data_dir, file_name)